In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import gc
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import plot_importance


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"botatu","key":"67288ace4f6e83162f50d94e0dab13eb"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
# !pip freeze
!pip install -q kaggle-cli

In [5]:
!kaggle competitions download -c talkingdata-adtracking-fraud-detection -f train_sample.csv
!kaggle competitions download -c talkingdata-adtracking-fraud-detection -f test.csv

  0% 0.00/1.08M [00:00<?, ?B/s]
100% 1.08M/1.08M [00:00<00:00, 71.8MB/s]
 92% 152M/165M [00:01<00:00, 127MB/s]
100% 165M/165M [00:01<00:00, 135MB/s]


In [6]:
!unzip train_sample.csv.zip
# !unzip train.csv.zip
!unzip test.csv.zip
# !unzip sample_submission.csv.zip
# !unzip test_supplement.csv.zip

Archive:  train_sample.csv.zip
replace mnt/ssd/kaggle-talkingdata2/competition_files/train_sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
!rm train_sample.csv.zip sample_submission.csv.zip test.csv.zip test_supplement.csv.zip train.csv.zip sample_submission.csv
!ls

rm: cannot remove 'sample_submission.csv.zip': No such file or directory
rm: cannot remove 'test_supplement.csv.zip': No such file or directory
rm: cannot remove 'train.csv.zip': No such file or directory
rm: cannot remove 'sample_submission.csv': No such file or directory
'kaggle (1).json'   kaggle.json   mnt   sample_data   test.csv


In [8]:
!ls mnt/ssd/kaggle-talkingdata2/competition_files/

train_sample.csv


In [0]:
# Add/update the count of a column at given index on a rolling basis
def update_count(index, data_till_now, train_data, col_name, count_col_name):
  col_value = train_data.iloc[index][col_name]
  prev_ip_rows = data_till_now[data_till_now[col_name]==col_value]
  if len(prev_ip_rows) > 0:
    train_data.at[index, count_col_name] = prev_ip_rows.iloc[-1][count_col_name] + 1
  else:
    train_data.at[index, count_col_name] = 1
  

def update_attribution_percentage(data_till_now, train_data, col_name, percentage_col_name, count_col_name):
  col_value = train_data.iloc[index][col_name]
  is_attributed_sum = data_till_now[data_till_now[col_name]==col_value]['is_attributed'].sum()
  if is_attributed_sum == 0:
    train_data.at[index, percentage_col_name] = 0
  else:
    train_data.at[index, percentage_col_name] = is_attributed_sum/(train_data.at[index, count_col_name])

In [28]:
dtypes = {  'ip': 'uint16',
            'app': 'uint16',
            'device': 'uint16',
            'os': 'uint16',
            'channel': 'uint16',
            'is_attributed': 'uint8'}

chunk_size=1000
train_data = pd.DataFrame()
start_index = 0
for chunk in pd.read_csv('mnt/ssd/kaggle-talkingdata2/competition_files/train_sample.csv', dtype=dtypes, parse_dates=['click_time', 'attributed_time'], chunksize=chunk_size):
  print("Processing chunk #{}".format(start_index/chunk_size))
  train_data = pd.concat([train_data, chunk], sort=False)
  for index, row in train_data[start_index:start_index+chunk_size].iterrows():
    data_till_now = train_data[:index]

    update_count(index, data_till_now, train_data, 'ip', 'clicks_by_ip')
    device_count = update_count(index, data_till_now, train_data, 'device', 'clicks_by_device')
    update_count(index, data_till_now, train_data, 'os', 'clicks_by_os')
    
    data_till_now = train_data[:index+1]
    
    update_attribution_percentage(data_till_now, train_data, 'ip', 'ip_click_attribution_percentage', 'clicks_by_ip')
    update_attribution_percentage(data_till_now, train_data, 'device', 'device_click_attribution_percentage', 'clicks_by_device')
    update_attribution_percentage(data_till_now, train_data, 'os', 'os_click_attribution_percentage', 'clicks_by_os')
  start_index+=chunk_size

Processing chunk #0.0
Processing chunk #1.0
Processing chunk #2.0
Processing chunk #3.0
Processing chunk #4.0
Processing chunk #5.0
Processing chunk #6.0
Processing chunk #7.0
Processing chunk #8.0
Processing chunk #9.0
Processing chunk #10.0
Processing chunk #11.0
Processing chunk #12.0
Processing chunk #13.0
Processing chunk #14.0
Processing chunk #15.0
Processing chunk #16.0
Processing chunk #17.0
Processing chunk #18.0
Processing chunk #19.0
Processing chunk #20.0
Processing chunk #21.0
Processing chunk #22.0
Processing chunk #23.0
Processing chunk #24.0
Processing chunk #25.0
Processing chunk #26.0
Processing chunk #27.0
Processing chunk #28.0
Processing chunk #29.0
Processing chunk #30.0
Processing chunk #31.0
Processing chunk #32.0
Processing chunk #33.0
Processing chunk #34.0
Processing chunk #35.0
Processing chunk #36.0
Processing chunk #37.0
Processing chunk #38.0
Processing chunk #39.0
Processing chunk #40.0
Processing chunk #41.0
Processing chunk #42.0
Processing chunk #43.

In [29]:
train_data['is_attributed'].value_counts()

0    99773
1      227
Name: is_attributed, dtype: int64

In [0]:
def update_latest_value_from_train_data(test_data, index, col_name, clicks_col_name, default_value):
  col_value = test_data.iloc[index][col_name]
  matching_train_data_rows = train_data[train_data[col_name] == col_value]
  if matching_train_data_rows.empty:
    test_data[clicks_col_name] = default_value
  else:
    test_data.at[index, clicks_col_name] = matching_train_data_rows[clicks_col_name].iloc[-1]

In [0]:
dtypes = {  'ip': 'uint16',
            'app': 'uint16',
            'device': 'uint16',
            'os': 'uint16',
            'channel': 'uint16'}

chunk_size=1000
test_data = pd.DataFrame()
start_index = 0
for chunk in pd.read_csv('test.csv', dtype=dtypes, parse_dates=['click_time'], chunksize=chunk_size):
  print("Processing chunk #{}".format(start_index/chunk_size))
  test_data = pd.concat([test_data, chunk], sort=False)
  for index, row in test_data[start_index:start_index+chunk_size].iterrows():
    update_latest_value_from_train_data(test_data, index, 'ip', 'clicks_by_ip', 1)
    update_latest_value_from_train_data(test_data, index, 'device', 'clicks_by_device', 1)
    update_latest_value_from_train_data(test_data, index, 'os', 'clicks_by_os', 1)
    
    update_latest_value_from_train_data(test_data, index, 'ip', 'ip_click_attribution_percentage', 0)
    update_latest_value_from_train_data(test_data, index, 'device', 'device_click_attribution_percentage', 0)
    update_latest_value_from_train_data(test_data, index, 'os', 'os_click_attribution_percentage', 0)
  start_index+=chunk_size
test_data.head()

Processing chunk #0.0
Processing chunk #1.0
Processing chunk #2.0
Processing chunk #3.0
Processing chunk #4.0
Processing chunk #5.0
Processing chunk #6.0
Processing chunk #7.0
Processing chunk #8.0
Processing chunk #9.0
Processing chunk #10.0
Processing chunk #11.0
Processing chunk #12.0
Processing chunk #13.0
Processing chunk #14.0
Processing chunk #15.0
Processing chunk #16.0
Processing chunk #17.0
Processing chunk #18.0
Processing chunk #19.0
Processing chunk #20.0
Processing chunk #21.0
Processing chunk #22.0
Processing chunk #23.0
Processing chunk #24.0
Processing chunk #25.0
Processing chunk #26.0
Processing chunk #27.0
Processing chunk #28.0
Processing chunk #29.0
Processing chunk #30.0
Processing chunk #31.0
Processing chunk #32.0
Processing chunk #33.0
Processing chunk #34.0
Processing chunk #35.0
Processing chunk #36.0
Processing chunk #37.0
Processing chunk #38.0
Processing chunk #39.0
Processing chunk #40.0
Processing chunk #41.0
Processing chunk #42.0
Processing chunk #43.

# Exploratory Data Analysis (EDA)

In [0]:
def percentage_plot(ax, height):
    ax.set_alpha(0.8)
    ax.tick_params(axis='both', which='both', length=0)

    total_height = 0
    for i in ax.patches:
        total_height += i.get_height()

    for i in ax.patches:
        ax.text(i.get_x(), i.get_height()+height, \
                str(round((i.get_height()/total_height)*100, 2))+'%', fontsize=15,
                    color='dimgrey')

In [0]:
fig, ax = plt.subplots()
plt.box(on=None)

cols = ['app', 'device', 'os', 'channel']
uniques = [len(train_data[col].unique()) for col in cols]

ax.bar(cols, uniques)
ax.set_xlabel('Number of unique values per feature')

percentage_plot(ax, 1.5)

In [0]:
fig, ax = plt.subplots()
plt.box(on=None)

mean = (train_data.is_attributed.values == 1).mean()
ax.bar(['Downloaded (1)', 'Not Downloaded (0)'], [mean, 1-mean])
ax.set(ylabel='Proportion', title='App Downloaded vs Not Downloaded')
percentage_plot(ax, 0.01)

In [0]:
plt.figure(figsize=(10,6))
plt.box(on=None)
plt.title('')



plt.bar(cols, uniques)

In [0]:
# is_attributed_true = train_data[train_data['is_attributed'] == 1]
# is_attributed_false = train_data[train_data['is_attributed'] == 0]

# grouped_data = train_data.groupby(['ip', 'is_attributed'], as_index=False).count().sort_values(by=['ip'], ascending=False)

# grouped_data.reset_index()
# grouped_data[]

In [0]:
plt.hist(train_data['ip'], bins=4)
plt.box(on=None)

The above chart tells us that IP with lesser values are used more. For example, IP values between range of 0 to 100000 were used the most. So the IPs seem to be sorted by how frequently they were used. Lets see how the IP to attribution ratio is in the below charts.

In [0]:
ip_with_attributed_time = train_data[train_data['attributed_time'].notna()]['ip']
plt.hist(ip_with_attributed_time, bins=4) 
plt.box(on=None)

print('Total unique IPs having attributed time = ', len(ip_with_attributed_time.unique()))


The maximum number of times an ad has been clicked from IPs that resulted in mobile download is 60.The previous graph tells us that IPs that did not result in app downloads were used more than 50,000 times!
Let us now see total number of times IPs that resulted in app download also did not result in download. That is, common IPs with and without attributed time.

In [0]:
# ip_without_attributed_time = train_data[train_data['attributed_time'].isna()]['ip']
# ips_with_both = pd.Series(list(set(ip_with_attributed_time) & set(ip_without_attributed_time)))

common_ips_with_and_without_attribution = train_data[train_data['ip'].isin(ip_with_attributed_time)]['ip']
plt.hist(common_ips_with_and_without_attribution, bins=4)

We can observe that the ips frequency that resulted in both attribution and non-attribution does not exceed 1600. From the above graphs, we can conclude that valid IPs for any dataset that result in successful downloads are not used more than 1600 times.

We might have to come up with similar comparison of IPs to os, app, device, channel, click_time to be able to put IPs into buckets better.

NOTES FOR ME: Condense results into single graph with 3 rows and add reusuable method for each field. Update results for undersampled training data.

```ip, app, device, os and channel``` are actually categorical variables encoded as integers. Set them as categories for analysis.

# Feature Engineering



In [0]:
def convert_to_categoricial(data, columns):
  for col_name in columns:
    data[col_name] = data[col_name].astype('category')

convert_to_categoricial(train_data, ['ip', 'app', 'device', 'os', 'channel', 'is_attributed'])

In [0]:
# Extracting time information

def extract_day_minute_hour(data, column_name):
  # Convert date stamps to date/time stamps
  data[column_name] = pd.to_datetime(data[column_name])

  data['day'] = data[column_name].dt.day.astype('uint8')
  data['hour'] = data[column_name].dt.hour.astype('uint8')
  data['minute'] = data[column_name].dt.minute.astype('uint8')
  data['second'] = data[column_name].dt.second.astype('uint8')
  


In [0]:
extract_day_minute_hour(train_data, 'click_time')
train_data.head()

In [0]:
extract_day_minute_hour(test_data, 'click_time')
convert_to_categoricial(test_data, ['ip', 'app', 'device', 'os', 'channel'])
test_data.head()

In [0]:
train_data.info()

In [0]:

test_data.info()

# Model Training

In [0]:

features = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'minute', 'second']


In [0]:
# The dataset is highly imbalanced
train_data['is_attributed'].value_counts().reset_index().replace({'index': {0:'not converted (can be fraud)', 1:'converted'}})

In [0]:
# Dealing with imbalanced datasets


In [0]:
clf = LogisticRegression().fit(train_data[features], train_data['is_attributed'])

In [0]:
# Predict the probability that the app was downloaded
test_data['is_attributed'] = clf.predict_proba(test_data[features])[:,1]

In [0]:
test_data['is_attributed'].head()

In [0]:
sub_cols = ['click_id', 'is_attributed']
test_data[sub_cols].to_csv('submission.csv', index=False)
!kaggle competitions submit -c talkingdata-adtracking-fraud-detection -f submission.csv -m "Test submission 3"